In [1]:
import tkinter as tk
import ttkbootstrap as ttk
from tkinter import messagebox
import pymongo
import datetime
from ttkbootstrap import Style
import pandas as pd
from tkinter import *
from ttkbootstrap.tableview import Tableview
from ttkbootstrap.constants import *

window = ttk.Window()
window.title('Salemba')
window.geometry('800x580')
window.resizable(False,False)

Style(theme="darkly")

################################################################################
#fungsi awal

def connect_to_mongodb():
    try:
        client = pymongo.MongoClient("mongodb://localhost:27017/")
        return client
    except pymongo.errors.ConnectionFailure:
        messagebox.showerror("Error", "Tidak dapat terhubung ke MongoDB.")
        return None

def clear_fields(*args):
    for entry in args:
        entry.delete(0, tk.END)

def logout():
    for widget in window.winfo_children():
        if widget != fr_login:
            widget.destroy()
    show_login_page()

def go_to_menu_page():
    for widget in window.winfo_children():
        if widget != fr_login:
            widget.destroy()    
    fr_login.pack_forget()
    main_page(window)

def frame_to_book():
    for widget in window.winfo_children():
        if widget != fr_login:
            widget.destroy()
    Menu_Buku(window)

def frame_to_supplier():
    for widget in window.winfo_children():
        if widget != fr_login:
            widget.destroy()
    Menu_Supplier(window)

def frame_to_pembeli():
    for widget in window.winfo_children():
        if widget != fr_login:
            widget.destroy()
    Menu_pembelian(window)

# Fungsi untuk menampilkan halaman login
def show_login_page():
    signup_frame.pack_forget()
    fr_login.pack(padx= 10, pady=40)

# Fungsi untuk menampilkan halaman sign up
def show_signup_page():
    fr_login.pack_forget()
    signup_frame.pack()
    
#####################################################################################
# fungsi get data untuk table

def get_supp_history():
    # Add data to the treeview
    client = connect_to_mongodb()
    db = client['salemba_app']

    pipeline = [
        {"$project": {"_id": 0, "Supplier": 1, "Alamat": 1, "No_Telp": 1}}
    ]

    hist = db['suppliers']
    historyy = hist.aggregate(pipeline)
    df = list(historyy)
    df = pd.DataFrame(df)
    # df.sort_values(by='tanggal', inplace = True)
    df.insert(0, 'No', range(1,len(df)+1))
    kol_list = df.columns.tolist()
    row_list = df.values.tolist()
    return df,kol_list, row_list

def get_book_history():
    # Add data to the treeview
    client = connect_to_mongodb()
    db = client['salemba_app']

    pipeline = [
        {"$project": {"_id": 0, "title": 1, "author": 1, "stock": 1, "supplier":1}}
    ]

    hist = db['books']
    historyy = hist.aggregate(pipeline)
    df = list(historyy)
    df = pd.DataFrame(df)
    # df.sort_values(by='tanggal', inplace = True)
    df.insert(0, 'No', range(1,len(df)+1))
    kol_list = df.columns.tolist()
    row_list = df.values.tolist()
    return df,kol_list, row_list

def get_pembeli_history():
    # Add data to the treeview
    client = connect_to_mongodb()
    db = client['salemba_app']

    pipeline = [
        {"$project": {"_id": 0, "nama_buku": 1, "nama_pembeli": 1, "jumlah_beli": 1}}
    ]

    hist = db['pembeli']
    historyy = hist.aggregate(pipeline)
    df = list(historyy)
    df = pd.DataFrame(df)
    df.insert(0, 'No', range(1,len(df)+1))
    kol_list = df.columns.tolist()
    row_list = df.values.tolist()
    return df,kol_list, row_list

####################################################################################3
# add data

def add_supplier():
    client = connect_to_mongodb()
    db = client['salemba_app']
    supplier_collection = db['suppliers'] 

    supp_name = entry_sup_name.get()
    street = entry_street.get()
    telp = entry_telp.get()

    # Check if the supplier with the same name already exists in the suppliers collection
    existing_supplier = supplier_collection.find_one({"Supplier": supp_name})
    if existing_supplier:
        messagebox.showerror("Error", f"Supplier dengan nama '{supp_name}' sudah terdaftar dalam database.")
        return

    if supp_name and street and telp:
        supplier_data = {
            "Supplier": supp_name,
            "Alamat": street,
            "No_Telp": int(telp)
        }
        supplier_collection.insert_one(supplier_data)
        messagebox.showinfo("Sukses", "Supplier berhasil ditambahkan ke database")
        clear_fields(entry_sup_name, entry_street, entry_telp)  
    else:
        messagebox.showerror("Error", "Semua kolom harus diisi!")

    frame_to_supplier()

def add_book():
    client = connect_to_mongodb()
    db = client['salemba_app']
    books_collection = db['books'] 

    title = entry_title.get()
    author = entry_author.get()
    stock = entry_stock.get()

    supplier = supplier_combobox.get()

    if not supplier:
        messagebox.showerror("Error", "Pilih supplier terlebih dahulu!")
        return

    # Check if the book with the same title already exists in the books collection
    existing_book = books_collection.find_one({"title": title})
    if existing_book:
        messagebox.showerror("Error", f"Buku dengan judul '{title}' sudah terdaftar dalam database.")
        return

    if title and author and stock:
        book_data = {
            "title": title,
            "author": author,
            "stock": int(stock),
            "supplier": supplier
        }
        books_collection.insert_one(book_data)
        messagebox.showinfo("Sukses", "Buku berhasil ditambahkan ke database")
        clear_fields(entry_title, entry_author, entry_stock)
    else:
        messagebox.showerror("Error", "Semua kolom harus diisi!")
    frame_to_book()

def submit():
    client = connect_to_mongodb()
    db = client['salemba_app']
    books_collection = db['books']
    pembeli_collection = db['pembeli']

    book_title = combobox_buku.get()
    pembeli = entry_nama.get()
    stock_to_buy = entry_stock.get()

    if not book_title or not pembeli or not stock_to_buy:
        messagebox.showerror("Error", "Semua kolom harus diisi.")
        return

    # Get the book document from the database based on the selected title
    book = books_collection.find_one({"title": book_title})

    if not book:
        messagebox.showerror("Error", f"Buku dengan judul '{book_title}' tidak ditemukan dalam database.")
        return

    current_stock = book["stock"]
    stock_to_buy = int(stock_to_buy)

    if stock_to_buy <= 0 or stock_to_buy > current_stock:
        messagebox.showerror("Error", f"Jumlah stock yang ingin dibeli harus lebih dari 0 dan tidak boleh melebihi stock yang tersedia.")
        return

    # Update the book's stock in the database
    new_stock = current_stock - stock_to_buy
    books_collection.update_one({"title": book_title}, {"$set": {"stock": new_stock}})

    # Insert the purchase data into the "pembeli" collection
    purchase_data = {
        "nama_buku": book_title,
        "nama_pembeli": pembeli,
        "jumlah_beli": stock_to_buy
    }
    pembeli_collection.insert_one(purchase_data)

    messagebox.showinfo("Sukses", f"Transaksi berhasil! Stock buku '{book_title}' tersisa {new_stock}.")
    # Refresh the book table to display the updated data
    frame_to_pembeli()

###########################################################################
# update data

def update_book():
    client = connect_to_mongodb()
    db = client['salemba_app']
    books_collection = db['books']

    title = entry_title.get()
    new_title = entry_title.get()
    author = entry_author.get()
    new_author = entry_author.get()
    stock = entry_stock.get()
    supplier = supplier_combobox.get()

    if not title:
        messagebox.showerror("Error", "Masukkan judul buku yang akan diupdate.")
        return

    # Check if the book with the given title exists in the books collection
    book = books_collection.find_one({"title": title})

    # Check if the book with the given author exists in the books collection
    book_by_author = books_collection.find_one({"author": author})

    # Update the book information in the books collection based on different scenarios
    book_data = {}
    if book and book_by_author:
        # If both book and book_by_author are found, update stock and supplier based on title and author
        book_data["stock"] = int(stock) if stock else book["stock"]
        book_data["supplier"] = supplier if supplier else book["supplier"]
        books_collection.update_one({"title": title}, {"$set": book_data})
        books_collection.update_one({"author": author}, {"$set": book_data})
    elif book:
        # If only book is found, update book information based on title
        book_data["title"] = new_title if new_title else title
        book_data["author"] = author
        book_data["stock"] = int(stock) if stock else book["stock"]
        book_data["supplier"] = supplier if supplier else book["supplier"]
        books_collection.update_one({"title": title}, {"$set": book_data})
    elif book_by_author:
        # If only book_by_author is found, update book information based on author
        book_data["title"] = title
        book_data["author"] = new_author if new_author else author
        book_data["stock"] = int(stock) if stock else book_by_author["stock"]
        book_data["supplier"] = supplier if supplier else book_by_author["supplier"]
        books_collection.update_one({"author": author}, {"$set": book_data})
    else:
        # If neither book nor book_by_author is found, show error message
        messagebox.showerror("Error", "Buku tidak ditemukan dalam database.")
        return

    messagebox.showinfo("Sukses", "Informasi buku berhasil diupdate di database.")
    # Refresh the book table to display the updated data
    frame_to_book()

def update_sup():
    client = connect_to_mongodb()
    db = client['salemba_app']
    supplier_collection = db['suppliers']
    books_collection = db['books']

    supp_name = entry_sup_name.get()
    street = entry_street.get()
    telp = entry_telp.get()

    if not supp_name and not street:
        messagebox.showerror("Error", "Masukkan nama supplier atau alamat yang akan diupdate.")
        return

    supplier = None
    supplier_by_street = None

    if supp_name:
        # Check if the supplier with the given name exists in the suppliers collection
        supplier = supplier_collection.find_one({"Supplier": supp_name})
    if street:
        # Check if the supplier with the given address exists in the suppliers collection
        supplier_by_street = supplier_collection.find_one({"Alamat": street})

    if supplier:
        # Update the supplier's information in the suppliers collection based on the given name
        supplier_data = {}
        if street:
            supplier_data["Alamat"] = street
        if telp:
            supplier_data["No_Telp"] = int(telp)

        supplier_collection.update_one({"Supplier": supp_name}, {"$set": supplier_data})

        # Check if there are books associated with the old supplier name
        # and update their supplier field with the new supplier name
        books_collection.update_many(
            {"supplier": supp_name},
            {"$set": {"supplier": supp_name}}
        )

        messagebox.showinfo("Sukses", f"Supplier dengan nama '{supp_name}' telah diupdate di database.")
        # Refresh the supplier table to display the updated data
        frame_to_supplier()

        # Refresh the book table to display the updated data with the new supplier name
        frame_to_book()

    elif supplier_by_street:
        # Update the supplier's information in the suppliers collection based on the given address
        supplier_data = {}
        if supp_name:
            supplier_data["Supplier"] = supp_name
        if telp:
            supplier_data["No_Telp"] = int(telp)

        supplier_collection.update_one({"Alamat": street}, {"$set": supplier_data})

        # Check if there are books associated with the old supplier name and address
        # and update their supplier field with the new supplier name
        books_collection.update_many(
            {"supplier": supplier_by_street["Supplier"], "Alamat": street},
            {"$set": {"supplier": supp_name}}
        )

        messagebox.showinfo("Sukses", f"Supplier dengan alamat '{street}' telah diupdate di database.")
        # Refresh the supplier table to display the updated data
        frame_to_supplier()

        # Refresh the book table to display the updated data with the new supplier name
        frame_to_book()

    else:
        messagebox.showerror("Error", "Supplier atau alamat tidak ditemukan dalam database.")
        return

##################################################################################
# Delete data

def del_book():
    client = connect_to_mongodb()
    db = client['salemba_app']
    books_collection = db['books']

    title = entry_title.get()

    if not title:
        messagebox.showerror("Error", "Masukkan judul buku yang akan dihapus.")
        return

    # Check if the book with the given title exists in the database
    book = books_collection.find_one({"title": title})
    if not book:
        messagebox.showerror("Error", "Buku tidak ditemukan dalam database.")
        return

    # Delete the book from the database
    books_collection.delete_one({"title": title})
    messagebox.showinfo("Sukses", f"Buku dengan judul '{title}' telah dihapus dari database.")
    # Refresh the book table to display the updated data
    frame_to_book()

def del_pembeli():
    client = connect_to_mongodb()
    db = client['salemba_app']
    pembeli_collection = db['pembeli']

    nama_pembeli = entry_nama.get()
    buku_dibeli = combobox_buku.get()

    if not nama_pembeli or not buku_dibeli:
        messagebox.showerror("Error", "Masukkan nama pembeli dan buku yang dibeli yang akan dihapus.")
        return

    # Check if the pembeli with the given name and book exists in the pembeli collection
    pembeli = pembeli_collection.find_one({"nama_pembeli": nama_pembeli, "nama_buku": buku_dibeli})
    if not pembeli:
        messagebox.showerror("Error", "Data pembeli tidak ditemukan dalam database.")
        return

    # Delete the pembeli from the database
    pembeli_collection.delete_one({"nama_pembeli": nama_pembeli, "nama_buku": buku_dibeli})
    messagebox.showinfo("Sukses", f"Data pembeli dengan nama '{nama_pembeli}' dan buku '{buku_dibeli}' telah dihapus dari database.")
    # Refresh the pembeli table to display the updated data
    frame_to_pembeli()

def del_supp():
    client = connect_to_mongodb()
    db = client['salemba_app']
    supplier_collection = db['suppliers']
    books_collection = db['books']

    supp_name = entry_sup_name.get()

    if not supp_name:
        messagebox.showerror("Error", "Masukkan nama supplier yang akan dihapus.")
        return

    # Check if the supplier with the given name exists in the database
    supplier = supplier_collection.find_one({"Supplier": supp_name})
    if not supplier:
        messagebox.showerror("Error", "Supplier tidak ditemukan dalam database.")
        return

    # Find all the books associated with the supplier
    books_to_delete = books_collection.find({"supplier": supp_name})

    # Delete the books associated with the supplier
    for book in books_to_delete:
        books_collection.delete_one({"_id": book["_id"]})

    # Delete the supplier from the database
    supplier_collection.delete_one({"Supplier": supp_name})
    messagebox.showinfo("Sukses", f"Supplier dengan nama '{supp_name}' dan buku-bukunya telah dihapus dari database.")
    # Refresh the supplier table to display the updated data
    frame_to_supplier()

####################################################################
# membaca data untuk combobox

def load_suppliers():
    client = connect_to_mongodb()
    db = client['salemba_app']
    supplier_collection = db['suppliers']
    suppliers = supplier_collection.distinct("Supplier")
    supplier_combobox["values"] = suppliers

def load_book():
    client = connect_to_mongodb()
    db = client['salemba_app']
    book_collection = db['books']
    books = book_collection.distinct("title")
    combobox_buku["values"] = books

#######################################################################
# Menu 

def Menu_pembelian(window):
    global combobox_buku, entry_nama, entry_stock
    Pembelian_fr =ttk.Frame(window, width = 500, height = 200)
    Pembelian_fr.pack()

    df, kol_list, row_list = get_pembeli_history()
    tabel_buku = Tableview(master= Pembelian_fr,
                        coldata=kol_list,
                        rowdata=row_list,
                        paginated=True,
                        pagesize=10,
                        autofit=True,
                        searchable=True,
                        )
    tabel_buku.pack(side= RIGHT, fill='both', expand=YES, padx=10, pady=10)
    tabel_buku.load_table_data()

    label_menu_pembeli = ttk.Label(Pembelian_fr, text='Menu Pembeli', font=('aileron', 18, 'bold'), justify='center')
    label_menu_pembeli.pack(pady=10)

    widgets_frame_pembeli = ttk.LabelFrame(Pembelian_fr, text="Insert Row")
    widgets_frame_pembeli.pack(padx=20, pady=10)

    Label_comb_pembeli = ttk.Label(widgets_frame_pembeli, text="Buku",)
    Label_comb_pembeli.pack()

    combobox_buku = ttk.Combobox(widgets_frame_pembeli, state="readonly",width=18)
    combobox_buku.pack(padx=10)

    load_book()

    label_nama = ttk.Label(widgets_frame_pembeli , text="Nama Pembeli")
    label_nama.pack(padx=10)

    entry_nama  = ttk.Entry(widgets_frame_pembeli, justify='center')
    entry_nama.pack(padx=10)

    label_stock = ttk.Label(widgets_frame_pembeli , text="Jumlah")
    label_stock.pack(padx=10)

    entry_stock = ttk.Entry(widgets_frame_pembeli, justify='center')
    entry_stock.pack(padx=10)

    button_submit_cus = ttk.Button(widgets_frame_pembeli, text="Submit", bootstyle ='success, Outline' , command= submit,
                            width= 19,cursor='hand2')
    button_submit_cus.pack(pady= 20,padx=10)

    button_del_cus = ttk.Button(Pembelian_fr, text="Delete", bootstyle ='danger, Outline',command=del_pembeli , width= 19,cursor='hand2')
    button_del_cus.pack(pady= 10,padx=10)

    button_back_to_menu = ttk.Button(Pembelian_fr, text="Back to Menu",command= go_to_menu_page ,width=22, style='secondary.TButton' ,cursor='hand2')
    button_back_to_menu.pack(pady=42)

def Menu_Supplier(window):
    global entry_sup_name, entry_street, entry_telp
    Supplier_fr = ttk.Frame(window, width = 500, height = 200)
    Supplier_fr.pack()

    df, kol_list, row_list = get_supp_history()
    tabel = Tableview(master= Supplier_fr,
                        coldata=kol_list,
                        rowdata=row_list,
                        paginated=True,
                        pagesize=10,
                        autofit=True,
                        searchable=True,
                        )
    tabel.pack(side= RIGHT, fill='both', expand=YES, padx=10, pady=10)
    tabel.load_table_data()

    label_menu_supplier = ttk.Label(Supplier_fr, text='Menu Supplier', font=('aileron', 18, 'bold'), justify='center')
    label_menu_supplier.pack(pady=10)

    widgets_frame_supp = ttk.LabelFrame(Supplier_fr, text="Insert Data")
    widgets_frame_supp.pack(padx=15,pady=15)
    
    label_sup_name = ttk.Label(widgets_frame_supp , text="Supplier Name")
    label_sup_name.pack(padx=10)

    entry_sup_name = ttk.Entry(widgets_frame_supp , justify='center')
    entry_sup_name.pack(padx=10)

    label_street = ttk.Label(widgets_frame_supp , text="Address")
    label_street.pack(padx=10)

    entry_street= ttk.Entry(widgets_frame_supp, justify='center')
    entry_street.pack(padx=10)

    label_telp = ttk.Label(widgets_frame_supp , text="Telp")
    label_telp.pack(padx=10)

    entry_telp = ttk.Entry(widgets_frame_supp,textvariable = tk.IntVar(), justify='center')
    entry_telp.pack(padx=10)

    button_add_supp = ttk.Button(widgets_frame_supp, text="Tambah Supplier", bootstyle ='success, Outline' , command= add_supplier, 
                            width= 19,cursor='hand2')
    button_add_supp.pack(pady= 10,padx=10)

    button_update_supp = ttk.Button(widgets_frame_supp, text="Update Supplier", bootstyle ='success, Outline' ,command= update_sup, 
                            width= 19,cursor='hand2')
    button_update_supp.pack(pady= 10,padx=10)

    button_del_supp = ttk.Button(Supplier_fr, text="Delete Supplier", bootstyle ='danger, Outline', command= del_supp, 
                            width= 19,cursor='hand2')
    button_del_supp.pack()

    button_back_to_menu = ttk.Button(Supplier_fr, text="Back to Menu",command= go_to_menu_page ,width= 25, style='secondary.TButton' ,cursor='hand2')
    button_back_to_menu.pack(pady=25)

def Menu_Buku(window):
    global entry_title, entry_author, entry_stock, supplier_combobox, tabel_buku

    Buku_fr =ttk.Frame(window)
    Buku_fr.pack()

    df, kol_list, row_list = get_book_history()
    tabel_buku = Tableview(master= Buku_fr,
                        coldata=kol_list,
                        rowdata=row_list,
                        paginated=True,
                        pagesize=10,
                        autofit=True,
                        searchable=True,
                        )
    tabel_buku.pack(side= RIGHT, fill='both', expand=YES, padx=10, pady=10)
    tabel_buku.load_table_data()

    label_menu_buku = ttk.Label(Buku_fr, text='Menu Buku', font=('aileron', 21, 'bold'), justify='center')
    label_menu_buku.pack()

    widgets_frame_buku = ttk.LabelFrame(Buku_fr, text="Insert Data")
    widgets_frame_buku.pack(padx=10)

    label_title = ttk.Label(widgets_frame_buku , text="Title")
    label_title.pack(padx=10)

    entry_title = ttk.Entry(widgets_frame_buku , justify='center')
    entry_title.pack(padx=10)

    label_author = ttk.Label(widgets_frame_buku , text="Author")
    label_author.pack(padx=10)

    entry_author = ttk.Entry(widgets_frame_buku, justify='center')
    entry_author.pack(padx=10)

    label_stock = ttk.Label(widgets_frame_buku , text="Stock")
    label_stock.pack(padx=10)

    entry_stock = ttk.Entry(widgets_frame_buku, justify='center')
    entry_stock.pack(padx=10)

    supplier_label = ttk.Label(widgets_frame_buku, text="Supplier",)
    supplier_label.pack(padx=10)

    supplier_combobox = ttk.Combobox(widgets_frame_buku, state="readonly",width=18)
    supplier_combobox.pack(padx=10)

    load_suppliers() 

    button_add_book = ttk.Button(widgets_frame_buku, text="Tambah Buku", bootstyle ='success, Outline' , command= add_book, 
                            width= 19,cursor='hand2')
    button_add_book.pack(pady= 10)

    button_update_book = ttk.Button(widgets_frame_buku, text="Update Buku", bootstyle ='success, Outline' , command= update_book, 
                            width= 19,cursor='hand2')
    button_update_book.pack(pady= 10)


    button_del_book = ttk.Button(Buku_fr, text="Delete Buku", bootstyle ='danger, Outline' ,command= del_book, width= 19,cursor='hand2')
    button_del_book.pack(pady= 10)

    button_back_to_menu = ttk.Button(Buku_fr, text="Back to Menu",command= go_to_menu_page ,width= 25, style='secondary.TButton' ,cursor='hand2')
    button_back_to_menu.pack()

def main_page(window):

    # Hide the login/signup page
    
    main_fr =ttk.Frame(window, width = 500, height = 200)
    main_fr.pack(pady=50)

    label_main_page = ttk.Label(main_fr, text='Main Menu Salemba App', font=('aileron', 18, 'bold'),anchor='center', width=500)
    label_main_page.pack(pady=50)

    button_book = ttk.Button(main_fr, text="Menu Buku", command = frame_to_book, width= 40,style='Primary.TButton',cursor='hand2' )
    button_book.pack(pady=10)
    
    button_supplier = ttk.Button (main_fr, text="Menu Supplier",command= frame_to_supplier, width= 40,style='Primary.TButton',cursor='hand2' )
    button_supplier.pack(pady=10)

    button_pembelian = ttk.Button(main_fr, text="Menu Pembelian",command= frame_to_pembeli, width= 40,style='Primary.TButton' ,cursor='hand2')
    button_pembelian.pack(pady=10)

    button_logout = ttk.Button(main_fr, text="Logout",command= logout, width= 40,style='secondary.TButton' ,cursor='hand2')
    button_logout.pack(pady=10)

def sign_up():
    name = name_entry.get()
    username = signup_username_entry.get()
    password = signup_password_entry.get()
    confirm_password = confirm_password_entry.get()

    if not name or not username or not password or not confirm_password:
        messagebox.showerror("Error", "Please fill all Data.")
        return
    
    if len(password) < 6:
        messagebox.showerror("Error", "Password must be more than 6 letters.")
        return
    
    if password != confirm_password:
        messagebox.showerror("Error", "Password and confirmation must be the same.")
        return

    # Menghubungkan ke MongoDB
    client = connect_to_mongodb()
    if client:
        db = client["salemba_app"]
        users_collection = db["users"]

        # Cek apakah username sudah terdaftar di database
        if users_collection.find_one({"username": username}):
            messagebox.showerror("Error", "Username already registered.")
            return

        tgl_hari_ini = datetime.date.today()
        tgl_format_tanggal = tgl_hari_ini.strftime("%d-%m-%Y")
        # Jika belum terdaftar, tambahkan ke database
        user_data = {
            "name": name,
            "username": username,
            "password": password,
            "join date": tgl_format_tanggal
        }
        users_collection.insert_one(user_data)
        messagebox.showinfo("Sukses", "Resigter Complete.")
        clear_fields()
        show_login_page()

def login():
    username = username_entry.get()
    password = password_entry.get()

    if not username or not password:
        messagebox.showerror("Error", "Please fill all Column.")
        return

    # Menghubungkan ke MongoDB
    client = connect_to_mongodb()
    if client:
        db = client["salemba_app"]
        users_collection = db["users"]

        # Cek apakah username dan password sesuai dengan data di database
        user = users_collection.find_one({"username": username, "password": password})
        if user:
            messagebox.showinfo("Sukses", "Login successful!")
            clear_fields(username_entry, password_entry)
            go_to_menu_page()
        else:
            messagebox.showerror("Error", "Incorrect username or password.")
            clear_fields(username_entry, password_entry)

##################################################################################################
#Login
# frame login
fr_login = ttk.Frame(window)
fr_login.pack(padx= 10, pady=40)

login_page_label2 = ttk.Label(fr_login, text="SALEMBA",font=('aileron', 33, 'bold'))
login_page_label2.pack(padx = 50, pady= 8)

login_page_label = ttk.Label(fr_login, text="Log into your Account",font=('Century Gothic',15))
login_page_label.pack(padx = 50, pady= 5)

# label username
login_username_label = ttk.Label(fr_login, text="Username")
login_username_label.pack(pady=10)

# entry username
username = tk.StringVar()
username_entry = ttk.Entry(fr_login, textvariable=username, justify='center')
username_entry.pack()

# label password
login_password_label = ttk.Label(fr_login, text="Password")
login_password_label.pack(pady=10)

# entry password
password = tk.StringVar()
password_entry = ttk.Entry(fr_login, textvariable=password, show='●', justify='center')
password_entry.pack()

# login button
login_button = ttk.Button(fr_login, text="LOGIN", bootstyle ='success, Outline' , command= login, 
                          width= 19,cursor='hand2')
login_button.pack(pady= 30)

# link Sign Up
signup_link = tk.Label(fr_login, text="Dont have an account? Sign Up",
                       fg="blue", cursor="hand2")
signup_link.pack()
signup_link.bind("<Button-1>", lambda event: show_signup_page())

###########################################################################################
# Sign Up
# Frame
signup_frame = ttk.Frame(window)
signup_frame.pack(pady= 30)

# label sign up
sign_page_label = ttk.Label(signup_frame, text="Sign Up",font=('aileron', 25, 'bold'))
sign_page_label.pack(pady= 20)

# label name
name_label = ttk.Label(signup_frame, text='Name')
name_label.pack(pady= 5)
# entry name
name_entry = tk.StringVar()
signup_name_entry = ttk.Entry(signup_frame, textvariable= name_entry, justify='center')
signup_name_entry.pack()

# label username
signup_username_label = ttk.Label(signup_frame, text="Username")
signup_username_label.pack(pady= 5)
# entry username
username = tk.StringVar()
signup_username_entry = ttk.Entry(signup_frame, textvariable=username, justify='center')
signup_username_entry.pack()

# label password
signup_password_label = ttk.Label(signup_frame, text="Password")
signup_password_label.pack(pady= 5)
# entry password
password = tk.StringVar()
signup_password_entry = ttk.Entry(signup_frame, textvariable=password, show='●',justify='center')
signup_password_entry.pack()

# Confirm password label
confirm_password_label = ttk.Label(signup_frame, text="Konfirmasi Password")
confirm_password_label.pack(pady= 5)
# entry confirm password
confirm_password_entry = ttk.Entry(signup_frame, show="●", justify='center')
confirm_password_entry.pack()

# login button
signup_button = ttk.Button(signup_frame, text="Sign Up", bootstyle ='success, Outline' , command=sign_up,width= 19,cursor='hand2')
signup_button.pack(pady= 25)


# link login
login_link = tk.Label(signup_frame, text="Have an Account? Login",
                      fg="blue", cursor="hand2")
login_link.pack(pady=15)
login_link.bind("<Button-1>", lambda event: show_login_page())

show_login_page()

window.mainloop()